In [1]:
import pprint
""" Import the GreenTranslator API wrapper. """
from greentranslator.api import GreenTranslator

""" Create an instance. """
translator = GreenTranslator ()

In [2]:
""" Query the exposure service. TODO: exercise other exposure endpoints. """
exposure = translator.\
    get_exposures().\
    get_exposure_by_area (exposure_type = 'pm25',
                        latitude      = '',
                        longitude     = '',
                        radius        = '0')
pprint.pprint (exposure, indent=2)

[ {'latitude': '35.7795897', 'longitude': '-78.6381787'},
  {'latitude': '35.9131996', 'longitude': '-79.0558445'},
  {'latitude': '35.9940329', 'longitude': '-78.898619'}]


In [3]:
""" 
3. What pediatric medications are available that act on the protein (gene) targets and biological pathways that are activated or inhibited by PM2.5 and ozone exposure?
  - This query will be extended to be a building block for this query.
  - Get drugs by condition. TODO: expand to specify a variety of related pheonotypes as parameters. 
"""
drugs = translator.\
    get_medbiochem().\
    get_drugs_by_condition (conditions=[ "d001249" ])
pprint.pprint (drugs, indent=2)

[ 'Tetracycline',
  'Topotecan',
  'Troglitazone',
  'Thiabendazole',
  'Torasemide',
  'Thiopental',
  'Trifluoperazine',
  'Theophylline',
  'Rosiglitazone',
  'Raloxifene',
  'Propranolol',
  'Fenofibrate',
  'Vitamin E',
  'Nitroglycerin',
  'Pentoxifylline',
  'Phenol',
  'Methimazole',
  'Phenylbutazone',
  'Phenobarbital',
  'Ethyl carbamate',
  'Acetaminophen',
  'Carmustine',
  'Zileuton',
  'Cilostazol',
  '1,2-Dihydroxybenzene',
  '2-(4-Morpholinyl)-8-Phenyl-4h-1-Benzopyran-4-One',
  '(11-BETA)-11,21-DIHYDROXY-PREGN-4-ENE-3,20-DIONE',
  '2H-1-BENZOPYRAN-2-ONE',
  'Melphalan',
  'Saquinavir',
  'Dimethylformamide',
  '3,4-Dihydroxycinnamic Acid',
  '(5e,13e)-9,15-Dihydroxy-11-Oxoprosta-5,13-Dien-1-Oicacid',
  'Glycerol',
  'Carbachol',
  'Imipramine',
  'Clomipramine',
  'Carbamyl-Choline',
  'Gefitinib',
  'Cyclophosphamide',
  'Fluorouracil',
  'Furosemide',
  'Doxorubicin',
  'Genistein',
  'Enflurane',
  'Melatonin',
  'Ethanol',
  'Iron',
  'Diethylcarbamodithioic Acid',

In [6]:
""" 
2. What protein (gene) targets and biological pathways are activated or inhibited by PM2.5 and ozone exposure?
   - This note is a building block for the second component of this query.
   - Get genes and pathways assoicated with a set of conditions. 
"""
conditions = [ 'd001249', 'd003371', 'd001249' ]
genes_paths = translator.\
    get_medbiochem().\
    get_genes_pathways_by_disease (diseases = conditions)
pprint.pprint (genes_paths[:3], indent=2)

[ { 'is_human': True,
    'kegg_path': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04666',
    'path_name': 'path:hsa04666 Fc gamma R-mediated phagocytosis - Homo '
                 'sapiens (human)',
    'uniprot_gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/MAPK3'},
  { 'is_human': True,
    'kegg_path': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04666',
    'path_name': 'path:hsa04666 Fc gamma R-mediated phagocytosis - Homo '
                 'sapiens (human)',
    'uniprot_gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/MAPK3'},
  { 'is_human': True,
    'kegg_path': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'path_name': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                 '(human)',
    'uniprot_gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'}]


In [4]:
""" Get conditions associated with an environmental stressor agent. """
exposures = translator.\
    get_medbiochem().\
    get_exposure_conditions (stressorAgentIDs = [ 'D052638' ])
pprint.pprint (exposures[:5], indent=2)

[ { 'conditionID': 'http://bio2rdf.org/mesh:D001247',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D005317',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D011297',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D011297',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D001724',
    'stressorAgentName': 'Particulate Matter'}]
